In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [3]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [4]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [5]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110259  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,9.275174e+06
2018-02-28,7.377206e+06
2018-03-31,9.084247e+06
2018-04-30,1.035057e+07
2018-05-31,1.093740e+07
2018-06-30,1.023005e+07
2018-07-31,1.093476e+07
2018-08-31,1.173815e+07
2018-09-30,9.697509e+06


In [6]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    9.275174e+06
 2018-02-28    7.377206e+06
 2018-03-31    9.084247e+06
 2018-04-30    1.035057e+07
 2018-05-31    1.093740e+07
 2018-06-30    1.023005e+07
 2018-07-31    1.093476e+07
 2018-08-31    1.173815e+07
 2018-09-30    9.697509e+06
 2018-10-31    1.040151e+07
 2018-11-30    9.272384e+06
 2018-12-31    8.326717e+06
 2019-01-31    9.408165e+06
 2019-02-28    8.526404e+06
 2019-03-31    9.545627e+06
 2019-04-30    1.032960e+07
 2019-05-31    1.123706e+07
 2019-06-30    1.044383e+07
 2019-07-31    1.165572e+07
 2019-08-31    1.226923e+07
 2019-09-30    9.172923e+06
 2019-10-31    1.106530e+07
 2019-11-30    1.046052e+07
 2019-12-31    1.037142e+07
 2020-01-31    1.058035e+07
 2020-02-29    7.650980e+06
 2020-03-31    7.422280e+06
 2020-04-30    7.325978e+06
 2020-05-31    8.936260e+06
 2020-06-30    9.016142e+06
 2020-07-31    1.028912e+07
 2020-08-31    1.195519e+07
 2020-09-30    8.431143e+06
 2020-10-31    1.072138e+07
 2020-11-30    1.075952e+07

# ARIMA

In [7]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [8]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.883633
p-value : 0.002159
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1306.037, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1300.298, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1301.915, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1296.790, Time=0.08 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1298.494, Time=0.12 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1298.069, Time=0.23 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1299.414, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1301.400, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1303.403, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1298.456, Time=0.14 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1300.340, Time=0.23 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1301.455, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1303.403, Time=0.09 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1301.544, Time=0.20 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1300.438, Time=0.21 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 3), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,0,3))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 3)   Log Likelihood                -643.333
Method:                       css-mle   S.D. of innovations        1062961.223
Date:                Sat, 11 Sep 2021   AIC                           1296.667
Time:                        01:43:08   BIC                           1305.355
Sample:                             0   HQIC                          1299.851
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.823e+06   3.76e+05     26.133      0.000    9.09e+06    1.06e+07
ma.L1.y        0.6344      0.151      4.213      0.000       0.339       0.930
ma.L2.y        0.2199      0.198      1.108      0.2

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([10780175.22041623,  9823372.51941428]), array([1062961.22312578, 1258841.59911581]), array([[ 8696809.50612706, 12863540.9347054 ],
       [ 7356088.32290648, 12290656.71592208]]))
